# First Run For Best Model

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve
from scipy.optimize import curve_fit
import tensorflow as tf
import keras
from keras import layers, models, regularizers
import pyts
from pyts.transformation.transformation import GADF
import json
import random
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import seaborn as sns

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Set a fixed seed
SEED = 42
set_seed(SEED)

# Create experiment directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_dir = os.path.join('experiments', f'experiment_{timestamp}')
os.makedirs(experiment_dir, exist_ok=True)
model_dir = os.path.join(experiment_dir, 'models')
results_dir = os.path.join(experiment_dir, 'results')
os.makedirs(model_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Define directories
data_dir = 'data'
synthetic_dir = os.path.join(data_dir, 'synthetic')
maps_dir = os.path.join(data_dir, 'maps')
labels_dir = os.path.join(data_dir, 'labels')
visualizations_dir = os.path.join(data_dir, 'visualizations')
problematic_spectra_dir = os.path.join(data_dir, 'problematic_spectra')
os.makedirs(synthetic_dir, exist_ok=True)
os.makedirs(maps_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(visualizations_dir, exist_ok=True)
os.makedirs(problematic_spectra_dir, exist_ok=True)

# Save experiment configuration
config = {
    'seed': SEED,
    'spectrum_length': 880,
    'image_size': 64,
    'num_synthetic_per_type': 999,
    'num_types': 11,
    'total_spectra': 11000,
    'epochs': 10,
    'batch_size_1d': 64,
    'batch_size_2d': 32,
    'validation_split': 0.1,
    'test_size': 0.2,
    'growth_rate': 12,
    'num_classes': 11,
    'experiment_timestamp': timestamp
}
with open(os.path.join(experiment_dir, 'config.json'), 'w') as f:
    json.dump(config, f, indent=4)

# Baseline functions
def poly_baseline(x, p, intensity, b):
    y = (x / len(x)) ** p + b
    return y * intensity / max(y)

def gaussian_baseline(x, mean, sd, intensity, b):
    y = np.exp(-(x - mean) ** 2 / (2 * sd ** 2)) / (sd * np.sqrt(2 * np.pi)) + b
    return y * intensity / max(y)

def pg_baseline(x, p, in1, mean, sd, in2, b):
    y1 = (x / len(x)) ** p + b
    y2 = np.exp(-(x - mean) ** 2 / (2 * sd ** 2)) / (sd * np.sqrt(2 * np.pi)) + b
    return y1 / max(y1) * in1 + y2 / max(y2) * in2

def mix_min_no(sp, baseline):
    return np.minimum(baseline, sp)

def iterative_fitting_with_bounds_no(sp, model, ite=10):
    fitted_baseline = np.zeros(sp.shape[0])
    x = np.linspace(1, sp.shape[0], sp.shape[0])
    tempb = sp
    torch_tempb = tf.expand_dims(tempb, axis=0)
    i = 0
    while i < ite:
        tadvice = model(torch_tempb)
        if tadvice[0][0] >= 0.5 and tadvice[0][1] >= 0.5:
            try:
                p, c = curve_fit(pg_baseline, x, tempb,
                                bounds=([1, 0.5, 0, 100, 0.5, -0.5], [3, 1, sp.shape[0], 600, 1, 0.5]),
                                maxfev=10000)
                fitted_baseline = pg_baseline(x, p[0], p[1], p[2], p[3], p[4], p[5])
            except RuntimeError:
                fitted_baseline = tempb
        elif tadvice[0][0] >= 0.5:
            try:
                p, c = curve_fit(poly_baseline, x, tempb,
                                bounds=([1, 0.5, -0.5], [3, 1, 0.5]),
                                maxfev=10000)
                fitted_baseline = poly_baseline(x, p[0], p[1], p[2])
            except RuntimeError:
                fitted_baseline = tempb
        elif tadvice[0][1] >= 0.5:
            try:
                p, c = curve_fit(gaussian_baseline, x, tempb,
                                bounds=([0, 100, 0.5, -0.5], [sp.shape[0], 600, 1, 0.5]),
                                maxfev=10000)
                fitted_baseline = gaussian_baseline(x, p[0], p[1], p[2], p[3])
            except RuntimeError:
                fitted_baseline = tempb
        tempb = mix_min_no(tempb, fitted_baseline)
        tempb_np = np.array(tempb)
        torch_tempb = tempb_np.reshape(1, sp.shape[0])
        i += 1
    return tempb

def create_baseline_model(input_shape=880):
    model = models.Sequential([
        layers.Input(shape=(input_shape,)),
        layers.Reshape((input_shape, 1)),
        layers.Conv1D(filters=16, kernel_size=5, strides=1, activation='relu'),
        layers.AveragePooling1D(pool_size=2, strides=2),
        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dense(2, activation='sigmoid')
    ])
    return model

def train_baseline_model(baseline_model, noise_data, epochs=10, batch_size=32):
    # Load labels from labels_noise_pure_182.npy
    try:
        labels = np.load(os.path.join(data_dir, 'labels_noise_pure_182.npy'))
        print("Đã tải nhãn từ labels_noise_pure_182.npy thành công!")
    except Exception as e:
        print(f"Lỗi khi tải nhãn: {e}. Sử dụng nhãn ngẫu nhiên.")
        labels = np.random.randint(0, 2, size=noise_data.shape[0])

    X = []
    y = []
    for i in range(noise_data.shape[0]):
        pure = noise_data[i, 0, 0, :, 0]
        noisy = noise_data[i, 0, 1, :, 0]
        X.append(noisy)
        y.append(labels[i])  # Sử dụng nhãn thực
    X = np.array(X)[:, :, np.newaxis]
    y = np.array(y)
    baseline_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    baseline_model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    baseline_model.save_weights(os.path.join(data_dir, 'model.weights.h5'))
    return baseline_model

# Normalize spectrum
def normalize_spectrum(spectrum):
    spectrum = spectrum - np.min(spectrum)
    if np.max(spectrum) > 0:
        spectrum = spectrum / np.max(spectrum)
    return spectrum

# WhittakerSmooth and airPLS
def WhittakerSmooth(x, w, lambda_=1, differences=1):
    X = np.matrix(x)
    m = X.size
    E = eye(m, format='csc')
    for i in range(differences):
        E = E[1:] - E[:-1]
    W = diags(w, 0, shape=(m, m))
    A = csc_matrix(W + (lambda_ * E.T * E))
    B = csc_matrix(W * X.T)
    background = spsolve(A, B)
    return np.array(background)

def airPLS(x, lambda_=100, porder=1, itermax=15):
    m = x.shape[0]
    w = np.ones(m)
    lambda_ = max(50, min(500, 50 * np.std(x) / (np.mean(np.abs(x)) + 1e-6)))
    for i in range(1, itermax + 1):
        z = WhittakerSmooth(x, w, lambda_, porder)
        d = x - z
        dssn = np.abs(d[d < 0].sum())
        if dssn < 0.001 * np.abs(x).sum():
            return z
        if i == itermax:
            print(f'WARNING: Max iteration reached! lambda_={lambda_:.2f}, dssn={dssn:.2e}')
            np.save(os.path.join(problematic_spectra_dir, f'problematic_spectrum_{np.random.randint(1000000)}.npy'), x)
            return WhittakerSmooth(x, np.ones(m), lambda_=50)
        w[d >= 0] = 0
        w[d < 0] = np.exp(i * np.abs(d[d < 0]) / dssn)
        w[0] = np.exp(i * (d[d < 0]).max() / dssn)
        w[-1] = w[0]
    return z

# Enhanced baseline correction
def enhanced_baseline_correction(spectrum, baseline_model):
    """
    Trừ nền: Dùng baseline_model để đoán và trừ nền sơ bộ (đa thức/Gaussian),
    sau đó dùng airPLS để tinh chỉnh. Nếu lỗi, dùng airPLS trực tiếp.
    """
    try:
        baseline = iterative_fitting_with_bounds_no(spectrum, baseline_model)
        fine_corrected = airPLS(baseline, lambda_=100, itermax=15)
        return np.clip(spectrum - fine_corrected, 0, None)
    except Exception as e:
        print(f"Error in baseline correction: {e}. Falling back to airPLS.")
        return np.clip(spectrum - airPLS(spectrum, lambda_=100, itermax=15), 0, None)

# Interpolate spectrum
def interpolate_spectrum(spectrum, original_length, target_length=880):
    x_original = np.linspace(0, original_length - 1, original_length)
    x_target = np.linspace(0, original_length - 1, target_length)
    interpolator = interp1d(x_original, spectrum, kind='linear', fill_value="extrapolate")
    return interpolator(x_target)

# Shift spectrum
def shift_spectrum(spectrum, shift):
    return np.roll(spectrum, shift)

# Stretch spectrum
def stretch_spectrum(spectrum, alpha):
    original_len = len(spectrum)
    new_len = int(original_len / alpha)
    if new_len < 1:
        new_len = 1
    if new_len > original_len * 10:
        new_len = original_len * 10
    x_original = np.linspace(0, original_len - 1, original_len)
    x_new = np.linspace(0, original_len - 1, new_len)
    interpolator = interp1d(x_original, spectrum, kind='linear', fill_value="extrapolate")
    stretched = interpolator(x_new)
    return interpolate_spectrum(stretched, new_len, original_len)

# Generate synthetic spectrum
def generate_synthetic_spectrum(input_spectrum, noise_data, spectrum_length=880):
    """
    Tạo phổ tổng hợp từ một phổ gốc duy nhất:
    - Thêm nhiễu từ noise_data và nhiễu Gaussian.
    - Thêm đường nền (đa thức/Gaussian hoặc không).
    - Dịch chuyển hoặc kéo dãn/nén.
    - Không trộn với phổ khác.
    """
    x_range = np.linspace(0, spectrum_length, spectrum_length)
    # Add noise from dataset
    noise_idx = np.random.randint(0, noise_data.shape[0])
    pure = noise_data[noise_idx, 0, 0, :, 0]
    noisy = noise_data[noise_idx, 0, 1, :, 0]
    noise = noisy - pure  # Shape (880,)
    scale = np.random.uniform(1.0, 2.0)
    synthetic_spectrum = input_spectrum + noise * scale
    # Add Gaussian noise
    synthetic_spectrum += np.random.normal(0, 0.05 * np.std(input_spectrum), spectrum_length)
    # Add synthetic baseline
    baseline_type = np.random.choice(['poly', 'gaussian', 'none'], p=[0.3, 0.3, 0.4])
    if baseline_type == 'poly':
        baseline = poly_baseline(x_range, p=np.random.uniform(1.9, 2.1),
                                intensity=np.random.uniform(0.75, 0.8),
                                b=np.random.uniform(-0.1, 0.1))
        synthetic_spectrum += baseline
    elif baseline_type == 'gaussian':
        baseline = gaussian_baseline(x_range, mean=np.random.uniform(0, spectrum_length),
                                   sd=np.random.uniform(250, 300),
                                   intensity=np.random.uniform(0.75, 0.8),
                                   b=np.random.uniform(-0.1, 0.1))
        synthetic_spectrum += baseline
    # Probabilistic augmentation
    aug_type = np.random.choice(['none', 'shift', 'stretch'], p=[0.5, 0.25, 0.25])
    if aug_type == 'shift':
        shift = np.random.randint(-10, 11)
        synthetic_spectrum = shift_spectrum(synthetic_spectrum, shift)
    elif aug_type == 'stretch':
        alpha = np.random.uniform(0.5, 2.0)
        synthetic_spectrum = stretch_spectrum(synthetic_spectrum, alpha)
    return synthetic_spectrum

# Create GADF map
def create_gadf_map(spectrum, image_size=64):
    spectrum = normalize_spectrum(spectrum)
    spectrum = 2 * spectrum - 1
    target_length = image_size * (spectrum.shape[0] // image_size)
    if target_length != spectrum.shape[0]:
        spectrum = interpolate_spectrum(spectrum, spectrum.shape[0], target_length)
    gadf = GADF(image_size=image_size, overlapping=False, scale='-1')
    return gadf.fit_transform(spectrum.reshape(1, -1))[0][:, :, np.newaxis]

# Load Excel data
def load_excel_data():
    excel_path = os.path.join(data_dir, 'Ethanol_Methanol.xlsx')
    try:
        df = pd.read_excel(excel_path, usecols='A:L')
        raman_shift = df['Raman Shift (cm-1)'].values
        spectra_data = {
            'ethanol': df['Ethanol'].values,
            'methanol': df['Methanol'].values,
            'EM1_a': df['EM1_a'].values,
            'EM2_a': df['EM2_a'].values,
            'EM3_a': df['EM3_a'].values,
            'EM4_a': df['EM4_a'].values,
            'EM5_a': df['EM5_a'].values,
            'EM6_a': df['EM6_a'].values,
            'EM7_a': df['EM7_a'].values,
            'EM8_a': df['EM8_a'].values,
            'EM9_a': df['EM9_a'].values
        }
        for key in spectra_data:
            spectrum = spectra_data[key]
            spectrum = spectrum[~np.isnan(spectrum)]
            spectra_data[key] = interpolate_spectrum(spectrum, len(spectrum), 880)
        return spectra_data, raman_shift
    except Exception as e:
        print(f"Lỗi khi tải dữ liệu từ {excel_path}: {e}")
        return {}, np.array([])

# Load noise data
def load_noise_data():
    try:
        # Noise data shape: (15000, 1, 2, 880, 1) - 15000 samples, 2 spectra (pure, noisy), 880 points
        noise_data = np.load(os.path.join(data_dir, 'dataset_noise_pure_182.npy'))
        return noise_data
    except Exception as e:
        print(f"Lỗi khi tải dữ liệu nhiễu: {e}")
        return np.array([])

# Process data
spectra_data, raman_shift = load_excel_data()
noise_data = load_noise_data()
if not spectra_data or noise_data.size == 0:
    raise FileNotFoundError("Không thể tải dữ liệu từ file Excel hoặc file nhiễu.")

# Initialize and train baseline model
baseline_model = create_baseline_model(input_shape=880)
try:
    baseline_model.load_weights(os.path.join(data_dir, 'model.weights.h5'))
    print("Trọng số mô hình baseline đã được tải thành công!")
except Exception as e:
    print(f"Lỗi khi tải trọng số: {e}. Training baseline model.")
    baseline_model = train_baseline_model(baseline_model, noise_data)

# Generate synthetic data
X_1d = []
X_2d = []
labels = []
sample_ids = []
example_spectra = {}
total_spectra = 0
spectrum_length = 880

ratio_to_label = {0.0: 0, 0.1: 1, 0.2: 2, 0.3: 3, 0.4: 4, 0.5: 5, 0.6: 6, 0.7: 7, 0.8: 8, 0.9: 9, 1.0: 10}
ratios = {
    'ethanol': 1.0,
    'methanol': 0.0,
    'EM1_a': 0.9,
    'EM2_a': 0.8,
    'EM3_a': 0.7,
    'EM4_a': 0.6,
    'EM5_a': 0.5,
    'EM6_a': 0.4,
    'EM7_a': 0.3,
    'EM8_a': 0.2,
    'EM9_a': 0.1
}

print("Bắt đầu sinh dữ liệu tổng hợp...")
for spectrum_type, ethanol_ratio in ratios.items():
    print(f"Xử lý {spectrum_type} với ethanol ratio {ethanol_ratio}...")
    input_spectrum = spectra_data[spectrum_type]
    normalized_raw = normalize_spectrum(input_spectrum)
    corrected_spectrum = enhanced_baseline_correction(normalized_raw, baseline_model)
    corrected_spectrum = normalize_spectrum(corrected_spectrum)
    X_1d.append(corrected_spectrum)
    X_2d.append(create_gadf_map(corrected_spectrum, image_size=64))
    label_idx = ratio_to_label[ethanol_ratio]
    labels.append(label_idx)
    sample_ids.append(f"{spectrum_type}_original")
    total_spectra += 1
    if total_spectra == 1 or ethanol_ratio not in example_spectra:
        example_spectra[ethanol_ratio] = {
            'raw': normalized_raw,
            'corrected': corrected_spectrum,
            'ethanol': spectra_data['ethanol'] if ethanol_ratio > 0 else None,
            'methanol': spectra_data['methanol'] if ethanol_ratio < 1 else None
        }
    for i in range(999):
        synthetic_spectrum = generate_synthetic_spectrum(normalized_raw, noise_data, spectrum_length)
        corrected_spectrum = enhanced_baseline_correction(synthetic_spectrum, baseline_model)
        corrected_spectrum = normalize_spectrum(corrected_spectrum)
        X_1d.append(corrected_spectrum)
        X_2d.append(create_gadf_map(corrected_spectrum, image_size=64))
        labels.append(label_idx)
        sample_ids.append(f"{spectrum_type}_synthetic_{i}")
        total_spectra += 1
        if i == 0:
            example_spectra[ethanol_ratio] = {
                'raw': synthetic_spectrum,
                'corrected': corrected_spectrum,
                'ethanol': spectra_data['ethanol'] if ethanol_ratio > 0 else None,
                'methanol': spectra_data['methanol'] if ethanol_ratio < 1 else None
            }
        if total_spectra % 1000 == 0:
            print(f"Đã xử lý {total_spectra} phổ.")

print(f"Tổng số phổ: {total_spectra}")

# Convert to numpy arrays
X_1d = np.array(X_1d)[:, :, np.newaxis]
X_2d = np.array(X_2d)
labels_df = pd.DataFrame({'label': labels, 'sample_id': sample_ids})

print("X_1d shape:", X_1d.shape)
print("X_2d shape:", X_2d.shape)
print("labels_df shape:", labels_df.shape)
print("Label distribution:\n", labels_df["label"].value_counts())

# Save data
labels_df.to_csv(os.path.join(labels_dir, "labels.csv"), index=False)
np.save(os.path.join(synthetic_dir, "synthetic_1d.npy"), X_1d)
np.save(os.path.join(maps_dir, "spectral_maps_gadf.npy"), X_2d)

# Plot spectra
def plot_spectra_comparison(wavenumbers, raw_spectrum, corrected_spectrum, ethanol_spectrum, methanol_spectrum, title, filename):
    plt.figure(figsize=(15, 10))
    plt.subplot(2, 1, 1)
    plt.plot(wavenumbers, raw_spectrum, label="Raw Spectrum", color='blue')
    plt.axvspan(1000, 1020, color='red', alpha=0.2, label="Vùng Methanol")
    plt.axvspan(870, 890, color='green', alpha=0.2, label="Vùng Ethanol")
    plt.xlabel("Bước sóng (cm⁻¹)")
    plt.ylabel("Cường độ chuẩn hóa")
    plt.title(f"Phổ Raw ({title})")
    plt.grid(True)
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(wavenumbers, corrected_spectrum, label="Corrected Spectrum", color='orange')
    if ethanol_spectrum is not None:
        plt.plot(wavenumbers, normalize_spectrum(ethanol_spectrum), label="Ethanol Component", color='green', linestyle='--')
    if methanol_spectrum is not None:
        plt.plot(wavenumbers, normalize_spectrum(methanol_spectrum), label="Methanol Component", color='red', linestyle='--')
    plt.axvspan(1000, 1020, color='red', alpha=0.2)
    plt.axvspan(870, 890, color='green', alpha=0.2)
    plt.xlabel("Bước sóng (cm⁻¹)")
    plt.ylabel("Cường độ chuẩn hóa")
    plt.title(f"Phổ Sau Trừ Nền và Thành Phần ({title})")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(visualizations_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Sử dụng phạm vi wavenumbers từ 500–3500 cm⁻¹
wavenumbers = np.linspace(500, 3500, 880)
for ratio in example_spectra:
    title = f"Ratio Ethanol/Methanol = {ratio:.2f}/{1-ratio:.2f}" if ratio < 1.0 else "Pure Ethanol" if ratio == 1.0 else "Pure Methanol"
    plot_spectra_comparison(
        wavenumbers,
        example_spectra[ratio]['raw'],
        example_spectra[ratio]['corrected'],
        example_spectra[ratio]['ethanol'],
        example_spectra[ratio]['methanol'],
        title,
        f"spectra_comparison_ratio_{ratio:.2f}.png"
    )

# Define DenseNet models
def build_1d_densenet(input_shape=(880, 1), num_classes=11, growth_rate=12):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(48, 7, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    def dense_block(x, num_layers, filters):
        for _ in range(num_layers):
            y = layers.BatchNormalization()(x)
            y = layers.Activation('relu')(y)
            y = layers.Conv1D(filters, 3, padding='same', kernel_regularizer=regularizers.l2(0.0005))(y)
            x = layers.Concatenate()([x, y])
        return x
    def transition_layer(x):
        filters = x.shape[-1]
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv1D(filters // 2, 1, padding='same', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        return x
    for _ in range(3):
        x = dense_block(x, num_layers=4, filters=growth_rate)
        x = transition_layer(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

def build_2d_densenet(input_shape=(64, 64, 1), num_classes=11, growth_rate=12):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(48, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    def dense_block(x, num_layers, filters):
        for _ in range(num_layers):
            y = layers.BatchNormalization()(x)
            y = layers.Activation('relu')(y)
            y = layers.Conv2D(filters, 3, padding='same', kernel_regularizer=regularizers.l2(0.0005))(y)
            x = layers.Concatenate()([x, y])
        return x
    def transition_layer(x):
        filters = x.shape[-1]
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters // 2, 1, padding='same', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        return x
    for _ in range(3):
        x = dense_block(x, num_layers=4, filters=growth_rate)
        x = transition_layer(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

# Define ResNet models
def build_1d_resnet(input_shape=(880, 1), num_classes=11):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, 5, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    def residual_block(x, filters, kernel_size=3):
        shortcut = x
        x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv1D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv1D(filters, 1, padding='same')(shortcut)
        x = layers.Add()([shortcut, x])
        x = layers.Activation('relu')(x)
        return x
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

def build_2d_resnet(input_shape=(64, 64, 1), num_classes=11):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    def residual_block(x, filters, kernel_size=3):
        shortcut = x
        x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.0005))(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(filters, kernel_size, padding='same')(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1, padding='same')(shortcut)
        x = layers.Add()([shortcut, x])
        x = layers.Activation('relu')(x)
        return x
    x = residual_block(x, 32)
    x = residual_block(x, 32)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title, filename):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=[f"{i*10}% Ethanol" for i in range(11)],
                yticklabels=[f"{i*10}% Ethanol" for i in range(11)])
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(os.path.join(visualizations_dir, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Data augmentation
data_augmentation_1d = models.Sequential([
    layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x), mean=0.0, stddev=0.05)),
    layers.Lambda(lambda x: x * tf.random.uniform((), 0.8, 1.2)),
    layers.Lambda(lambda x: tf.roll(x, shift=tf.random.uniform((), -5, 5, dtype=tf.int32), axis=1))
])
data_augmentation_2d = models.Sequential([
    layers.Lambda(lambda x: x + tf.random.normal(tf.shape(x), mean=0.0, stddev=0.05)),
    layers.Lambda(lambda x: x * tf.random.uniform((), 0.8, 1.2)),
    layers.Lambda(lambda x: tf.roll(x, shift=tf.random.uniform((), -5, 5, dtype=tf.int32), axis=1))
])

# Split data
y = labels_df["label"].values
X_1d_train, X_1d_test, y_train, y_test = train_test_split(X_1d, y, test_size=0.2, random_state=42)
X_2d_train, X_2d_test, y_train_2d, y_test_2d = train_test_split(X_2d, y, test_size=0.2, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.arange(11), y=y)
class_weight = {i: w for i, w in enumerate(class_weights)}

# Train models with fresh optimizers
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# DenseNet 1D
tf.keras.backend.clear_session()
lr_schedule_1d_densenet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_1d_train)//64)
optimizer_1d_densenet = keras.optimizers.Adam(learning_rate=lr_schedule_1d_densenet)
densenet_1d = models.Sequential([data_augmentation_1d, build_1d_densenet()])
densenet_1d.compile(optimizer=optimizer_1d_densenet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
densenet_1d.fit(X_1d_train, y_train, validation_split=0.1, epochs=10, batch_size=64,
                callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_densenet_1d.keras"), save_best_only=True), early_stopping],
                class_weight=class_weight)

# DenseNet 2D
tf.keras.backend.clear_session()
lr_schedule_2d_densenet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_2d_train)//32)
optimizer_2d_densenet = keras.optimizers.Adam(learning_rate=lr_schedule_2d_densenet)
densenet_2d = models.Sequential([data_augmentation_2d, build_2d_densenet()])
densenet_2d.compile(optimizer=optimizer_2d_densenet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
densenet_2d.fit(X_2d_train, y_train_2d, validation_split=0.1, epochs=10, batch_size=32,
                callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_densenet_2d.keras"), save_best_only=True), early_stopping],
                class_weight=class_weight)

# ResNet 1D
tf.keras.backend.clear_session()
lr_schedule_1d_resnet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_1d_train)//64)
optimizer_1d_resnet = keras.optimizers.Adam(learning_rate=lr_schedule_1d_resnet)
resnet_1d = models.Sequential([data_augmentation_1d, build_1d_resnet()])
resnet_1d.compile(optimizer=optimizer_1d_resnet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet_1d.fit(X_1d_train, y_train, validation_split=0.1, epochs=10, batch_size=64,
              callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_resnet_1d.keras"), save_best_only=True), early_stopping],
              class_weight=class_weight)

# ResNet 2D
tf.keras.backend.clear_session()
lr_schedule_2d_resnet = keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10*len(X_2d_train)//32)
optimizer_2d_resnet = keras.optimizers.Adam(learning_rate=lr_schedule_2d_resnet)
resnet_2d = models.Sequential([data_augmentation_2d, build_2d_resnet()])
resnet_2d.compile(optimizer=optimizer_2d_resnet, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
resnet_2d.fit(X_2d_train, y_train_2d, validation_split=0.1, epochs=10, batch_size=32,
              callbacks=[keras.callbacks.ModelCheckpoint(os.path.join(model_dir, "best_resnet_2d.keras"), save_best_only=True), early_stopping],
              class_weight=class_weight)

# Evaluate
y_pred_1d_densenet = densenet_1d.predict(X_1d_test)
y_pred_2d_densenet = densenet_2d.predict(X_2d_test)
y_pred_1d_resnet = resnet_1d.predict(X_1d_test)
y_pred_2d_resnet = resnet_2d.predict(X_2d_test)

y_pred_1d_densenet_labels = np.argmax(y_pred_1d_densenet, axis=1)
y_pred_2d_densenet_labels = np.argmax(y_pred_2d_densenet, axis=1)
y_pred_1d_resnet_labels = np.argmax(y_pred_1d_resnet, axis=1)
y_pred_2d_resnet_labels = np.argmax(y_pred_2d_resnet, axis=1)

densenet_1d_acc = np.mean(y_pred_1d_densenet_labels == y_test)
densenet_2d_acc = np.mean(y_pred_2d_densenet_labels == y_test_2d)
resnet_1d_acc = np.mean(y_pred_1d_resnet_labels == y_test)
resnet_2d_acc = np.mean(y_pred_2d_resnet_labels == y_test_2d)

densenet_1d_precision = precision_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_precision = precision_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_precision = precision_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_precision = precision_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

densenet_1d_recall = recall_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_recall = recall_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_recall = recall_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_recall = recall_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

densenet_1d_f1 = f1_score(y_test, y_pred_1d_densenet_labels, average='macro')
densenet_2d_f1 = f1_score(y_test_2d, y_pred_2d_densenet_labels, average='macro')
resnet_1d_f1 = f1_score(y_test, y_pred_1d_resnet_labels, average='macro')
resnet_2d_f1 = f1_score(y_test_2d, y_pred_2d_resnet_labels, average='macro')

print("\nKết quả đánh giá:")
print(f"DenseNet 1D - Accuracy: {densenet_1d_acc:.4f}, Precision: {densenet_1d_precision:.4f}, Recall: {densenet_1d_recall:.4f}, F1: {densenet_1d_f1:.4f}")
print(f"DenseNet 2D (GADF) - Accuracy: {densenet_2d_acc:.4f}, Precision: {densenet_2d_precision:.4f}, Recall: {densenet_2d_recall:.4f}, F1: {densenet_2d_f1:.4f}")
print(f"ResNet 1D - Accuracy: {resnet_1d_acc:.4f}, Precision: {resnet_1d_precision:.4f}, Recall: {resnet_1d_recall:.4f}, F1: {resnet_1d_f1:.4f}")
print(f"ResNet 2D (GADF) - Accuracy: {resnet_2d_acc:.4f}, Precision: {resnet_2d_precision:.4f}, Recall: {resnet_2d_recall:.4f}, F1: {resnet_2d_f1:.4f}")

# Save results
plot_confusion_matrix(y_test, y_pred_1d_densenet_labels, "Confusion Matrix - DenseNet 1D", "cm_densenet_1d.png")
plot_confusion_matrix(y_test_2d, y_pred_2d_densenet_labels, "Confusion Matrix - DenseNet 2D (GADF)", "cm_densenet_2d_gadf.png")
plot_confusion_matrix(y_test, y_pred_1d_resnet_labels, "Confusion Matrix - ResNet 1D", "cm_resnet_1d.png")
plot_confusion_matrix(y_test_2d, y_pred_2d_resnet_labels, "Confusion Matrix - ResNet 2D (GADF)", "cm_resnet_2d_gadf.png")

densenet_1d.save(os.path.join(model_dir, 'densenet_1d_full.keras'))
densenet_2d.save(os.path.join(model_dir, 'densenet_2d_full.keras'))
resnet_1d.save(os.path.join(model_dir, 'resnet_1d_full.keras'))
resnet_2d.save(os.path.join(model_dir, 'resnet_2d_full.keras'))

np.save(os.path.join(results_dir, 'y_pred_1d_densenet.npy'), y_pred_1d_densenet)
np.save(os.path.join(results_dir, 'y_pred_2d_densenet.npy'), y_pred_2d_densenet)
np.save(os.path.join(results_dir, 'y_pred_1d_resnet.npy'), y_pred_1d_resnet)
np.save(os.path.join(results_dir, 'y_pred_2d_resnet.npy'), y_pred_2d_resnet)

metrics = {
    'densenet_1d': {'accuracy': densenet_1d_acc, 'precision': densenet_1d_precision, 'recall': densenet_1d_recall, 'f1': densenet_1d_f1},
    'densenet_2d': {'accuracy': densenet_2d_acc, 'precision': densenet_2d_precision, 'recall': densenet_2d_recall, 'f1': densenet_2d_f1},
    'resnet_1d': {'accuracy': resnet_1d_acc, 'precision': resnet_1d_precision, 'recall': resnet_1d_recall, 'f1': resnet_1d_f1},
    'resnet_2d': {'accuracy': resnet_2d_acc, 'precision': resnet_2d_precision, 'recall': resnet_2d_recall, 'f1': resnet_2d_f1}
}
with open(os.path.join(results_dir, 'metrics.json'), 'w') as f:
    json.dump(metrics, f, indent=4)